In [ ]:
import time
start = time.time()

In [ ]:
import concurrent.futures
import json
import os
import re
import sys
import typing as T
import plotly.express as px
import plotly.graph_objects as go
from threading import Lock

import dotenv
import googlemaps
import nltk
import pandas as pd
from geopy.geocoders import Nominatim
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from openai import OpenAI

dotenv.load_dotenv()

# Secrets
GOOGLE_API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")
assert GOOGLE_API_KEY is not None, "GOOGLE_PLACES_API_KEY is not set"
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
assert OPEN_AI_API_KEY is not None, "OPEN_AI_API_KEY is not set"
MAPBOX_API_KEY = os.getenv("MAPBOX_API_KEY")
assert MAPBOX_API_KEY is not None, "MAPBOX_API_KEY is not set"

CURRENT_DIR = %pwd
ROOT_DIR = os.path.dirname(CURRENT_DIR)
SRC_DIR = os.path.join(ROOT_DIR, "src")

sys.path.append(SRC_DIR)

In [ ]:
nltk.download("averaged_perceptron_tagger")
nltk.download("punkt")
nltk.download("stopwords")

gmaps = googlemaps.Client(key=GOOGLE_API_KEY)
px.set_mapbox_access_token(MAPBOX_API_KEY)

TYPES = [
    "restaurant", # default
    "bakery",
    "sandwich_shop",
    "coffee_shop",
    "cafe",
    "fast_food_restaurant",
    "store",
    "food",
    "point_of_interest",
    "establishment",
]

DEFAULT_TYPE = TYPES[0]

In [ ]:
METERS_PER_MILE = 1609.34


def miles_to_meters(miles: float) -> float:
    return miles * METERS_PER_MILE


def get_city_center_coordinates(city_name: str) -> T.Optional[T.Tuple[float, float]]:
    geolocator = Nominatim(user_agent="tgtg")

    location = geolocator.geocode(city_name)

    if not location:
        return None

    return (location.latitude, location.longitude)


def clean_text(text: str) -> str:
    """
    Function to remove stop words, punctuation, and double quotes
    """
    text = text.lower()
    text = text.replace('"', "")
    # Tokenize the text into words
    words = word_tokenize(text)
    # Remove punctuation
    words = [word for word in words if word.isalnum()]
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word not in stop_words]
    cleaned_text = " ".join(filtered_words)
    return cleaned_text


def parse_itenerary_day(lines: T.List[str]) -> T.List[T.Tuple[str, str]]:
    day_plan = []
    # This pattern is designed to capture two groups separated by various delimiters
    pattern = re.compile(r"\-?\s*([\w\s]+?)\s*(?::|at|-)\s*([\w\s'&]+)")

    for line in lines:
        match = pattern.match(line)
        if match:
            activity_type, place = match.groups()
            activity_type = activity_type.strip()
            place = place.split("(")[0].strip()  # Removes anything within parentheses
            day_plan.append((activity_type, place))
        else:
            print(f"Could not parse line: {line}")

    return day_plan


def parse_itenerary_content(content: str) -> T.List[T.Dict[str, str]]:
    # Split the content into days and activities
    days = content.split("\n\n")
    data = []
    for day in days:
        lines = day.split("\n")
        day_number = lines[0].split(" ")[1]
        day_plan = parse_itenerary_day(lines[1:])
        for activity_type, place in day_plan:
            data.append({"Day": day_number, "Activity Type": activity_type, "Place": place})

    return data

In [ ]:
inputs = {
    "location": "South Beach Miami, FL",
    "number_of_people": 6,
    "date": "November 2026",
    "duration_days": 3,
    "group_type": "bachelorette party",
    "description": (
        "include boutique hotel options must 4 stars higher onsite spa beach clubs djs"
        "day high end nightclubs list least six restaurant options dinner include least "
        "one nice steakhouse one nice sushi restaurant"
    ),
}

In [ ]:
PROMPT_START = """Create itinerary: provide only names
for places listed in google places for breakfast,
morning activity, lunch, afternoon activity, dinner and
evening activity. return only the day and the name:
""".replace(
    "\n", " "
)

PROMPT_TEMPLATE = f"""“{inputs['duration_days']}” day
“{inputs['group_type']}” to “{inputs['location']}” for “{inputs['number_of_people']}”
people in “{inputs['date']}” that enjoy “{inputs['description']}”
"""

prompt = f"{PROMPT_START}{clean_text(PROMPT_TEMPLATE)}"
print(prompt)

In [ ]:
client = OpenAI(api_key=OPEN_AI_API_KEY)


response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)
print(response.choices[0].message.content.split("\n"))

# Extract token usage from the response
completion_tokens = response.usage.completion_tokens
prompt_tokens = response.usage.prompt_tokens
total_tokens = completion_tokens + prompt_tokens

# Cost estimation for 1,000,000 requests
total_requests = 1_000_000
total_tokens_for_all_requests = total_tokens * total_requests

training_cost_per_million_tokens = 8.00
input_usage_cost_per_million_tokens = 3.00
output_usage_cost_per_million_tokens = 6.00

total_cost = (total_tokens_for_all_requests / 1_000_000) * (
    training_cost_per_million_tokens
    + input_usage_cost_per_million_tokens
    + output_usage_cost_per_million_tokens
)

print(f"Estimated total cost for {total_requests} requests: ${total_cost:.2f}")

In [ ]:
# Extract the itinerary content from the response
itinerary_content = response.choices[0].message.content
data = parse_itenerary_content(itinerary_content)
df = pd.DataFrame(data)

print(df)

In [ ]:
# Inputs for the nearby lookup, if store_type is left as None, it will
# default to match the type of the place that we are searching nearby from
keyword = "breakfast"
store_type = "restaurant"
radius_meters = 1500
# radius_from_place_miles = 1.0
# radius_meters = miles_to_meters(radius_from_place_miles)

In [ ]:
city_coordinates = get_city_center_coordinates(inputs["location"])
print(f"{inputs['location']} coordinates: {city_coordinates}")

assert store_type in TYPES, f"Invalid store type: {store_type}, must be one of {','.join(TYPES)}"

itinerary_place_details = []
nearby_place_details = {}

lock = Lock()


def get_place_details(row, store_type=None):
    try:
        result = gmaps.places(query=row["Place"], location=city_coordinates)
    except:
        print(f"Unable to get places info for {row['Place']}")
        return None

    if not result or result.get("status") != "OK":
        print(f"Unable to get places info for {row['Place']}")
        return None

    place_result = result["results"][0]
    with lock:
        itinerary_place_details.append((row["Place"], place_result))

    store_type = store_type if store_type else place_result.get("types", [DEFAULT_TYPE])[0]

    print(f"Getting nearby places for {row['Place']} at {place_result['geometry']['location']}")
    try:
        nearby_places = gmaps.places_nearby(
            location=place_result["geometry"]["location"],
            radius=radius_meters,
            keyword=keyword if keyword else None,
            type=store_type,
        )
    except:
        print(f"Unable to get nearby places info for {row['Place']}")
        return None

    if not nearby_places or nearby_places.get("status") != "OK":
        print(f"Unable to get nearby places info for {row['Place']}")
        return None

    with lock:
        nearby_place_details[row["Place"]] = nearby_places["results"]

    print(f"Found {len(nearby_places['results'])} nearby places for {row['Place']}")
    return None


with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(get_place_details, row, store_type) for _, row in df.iterrows()]
    concurrent.futures.wait(futures)

In [ ]:
print(f"Num results: {len(itinerary_place_details)}")
for place, item in itinerary_place_details:
    print(item["name"])
    nearby_names = [i["name"] for i in nearby_place_details.get(place, [])]
    for name in nearby_names:
        print(f"  - {name}")

In [ ]:
total_time = time.time() - start

print(f"Total time taken: {total_time:.2f} seconds")

In [ ]:
# Create a map showing the places in the itinerary and nearby places
print("Creating map...")

MAX_NEARBY_PLACES = 3

places_dfs = {}
color = 0
color_increment = 1.0 / len(itinerary_place_details)
for name, place in itinerary_place_details:
    color += color_increment
    rows = [(place["name"], *place["geometry"]["location"].values(), color)]
    max_clamp = min(MAX_NEARBY_PLACES, len(nearby_place_details.get(name, [])))
    rows.extend(
        [
            (x["name"], *x["geometry"]["location"].values(), color)
            for x in nearby_place_details.get(name, [])[:max_clamp]
        ]
    )
    places_dfs[name] = pd.DataFrame(rows, columns=["name", "latitude", "longitude", "color"])


fig = go.Figure()

for row in df.iterrows():
    places_coords_df = places_dfs[row[1]["Place"]]
    trace_name = f"Day {row[1]['Day']}: {row[1]['Place']}"
    fig.add_trace(
        go.Scattermapbox(
            lat=places_coords_df["latitude"],
            lon=places_coords_df["longitude"],
            mode="markers",
            marker=go.scattermapbox.Marker(
                size=5, color=places_coords_df["color"], colorscale="Rainbow", cmin=0, cmax=1
            ),
            text=places_coords_df["name"],
            name=trace_name,
        )
    )

fig.update_layout(
    autosize=True,
    hovermode="closest",
    mapbox=go.layout.Mapbox(
        accesstoken=os.getenv("MAPBOX_API_KEY"),
        bearing=0,
        center=go.layout.mapbox.Center(lat=city_coordinates[0], lon=city_coordinates[1]),
        pitch=0,
        zoom=12,
    ),
    height=800,
    width=1000,
    title_text=f"Itinerary for `{inputs['group_type']}` in {inputs['location']}",
    title_x=0.5,
    annotations=[
        dict(
            text=f"Nearby places for `{keyword}` near the places in the itinerary",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.5,
            y=0.0,
            xanchor="center",
            yanchor="top",
            font=dict(size=14, color="black"),
        )
    ],
)

fig.show()